In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import seaborn as sns
import gc
from tqdm.notebook import tqdm
import warnings

warnings.filterwarnings('ignore')

%matplotlib inline

PATH = '../'

In [2]:
%%time
train = pd.read_csv(PATH + 'train.csv')
test = pd.read_csv(PATH + 'Track 1/test.csv')
submission = pd.read_csv(PATH + 'Track 1/submission.csv')

CPU times: user 1.69 s, sys: 180 ms, total: 1.87 s
Wall time: 1.87 s


In [3]:
def smape(satellite_predicted_values, satellite_true_values): 
    # the division, addition and subtraction are pointwise 
    return np.mean(np.abs(satellite_predicted_values - satellite_true_values) / (np.abs(satellite_predicted_values) + np.abs(satellite_true_values)))

In [4]:
training_columns = ['epoch', 'sat_id', 'x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']
prediction_columns = ['x','y','z','Vx', 'Vy', 'Vz', 'epoch']
X = train.loc[:, training_columns]
y = train.loc[:, prediction_columns]
Xtest = test.loc[:, training_columns]

In [16]:
##moell sarima add roliing
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

smp_total = []
for sat_id in tqdm(Xtest['sat_id'].unique()):
    satX = X[X['sat_id'] == sat_id]
    satY = y[X['sat_id'] == sat_id]
    satXtest = Xtest[Xtest['sat_id'] == sat_id]
    
    trainSize = 0.6
    size = int(satX.shape[0] * trainSize)
    Xtr, Xval = satX.iloc[:size, :], satX.iloc[size:, :]
    ytr, yval = satY.iloc[:size, :], satY.iloc[size:, :]
    smp_sat = []
    for j in range(6):
        ##train part
        smp = []
        try:
            target = ytr.iloc[:, j]
            yvalj = yval.iloc[:, j]
            li = [(target.iloc[0+i:-24+i]).reset_index(drop=True).rename('r{}'.format(24-i)) for i in range(24)]
            mean = target.rolling(24).apply(lambda x: x.mean()).rename('mean').iloc[23:-1].reset_index(drop=True)
            std = target.rolling(24).apply(lambda x: x.std()).rename('std').iloc[23:-1].reset_index(drop=True)
            li.append(mean)
            li.append(std)
            x = pd.concat(li, axis = 1)
            ya = target.shift(-24).iloc[:-24].reset_index(drop=True)
            from sklearn.linear_model import Ridge
            ridge = Ridge(alpha = 0.001, normalize=False)
            ridge.fit(x, ya)
            for i in range(len(yvalj)):
                mean = target.tail(25).rolling(24).apply(lambda x: x.mean()).rename('mean').iloc[-1]
                std = target.tail(25).rolling(24).apply(lambda x: x.std()).rename('std').iloc[-1]
                li = [target.iloc[-24+i] for i in range(24)]
                li.append(mean)
                li.append(std)
                x = np.array(li).reshape(1, len(li))
                ya = pd.Series(ridge.predict(x))
                ya.index = [target.index[-1]+1]
                target = target.append(pd.Series(ya))
            smp.append(smape(target[-yvalj.shape[0]:], yvalj))
        except Exception:
            smp.append(0)
        try:
            try:
                mod = sm.tsa.statespace.SARIMAX(ytr.iloc[:,j], order=(0,0,0), seasonal_order=(1,0,1,24), simple_differencing=True)
                res = mod.fit(disp=False)
            except Exception as e:
                print()
                try:
                    mod = sm.tsa.statespace.SARIMAX(ytr.iloc[:,j], order=(0,0,0), seasonal_order=(1,0,0,24), simple_differencing=True)
                    res = mod.fit(disp=False)
                except Exception as e:
                    mod = sm.tsa.statespace.SARIMAX(ytr.iloc[:,j], order=(0,0,0), seasonal_order=(1,0,2,24), simple_differencing=True)
                    res = mod.fit(disp=False)

            nforecast = satX.shape[0] - size
            pred = res.get_forecast(steps = nforecast)
            ypred = pred.predicted_mean
            assert size+len(ypred) == satY.shape[0]
            assert len(ypred) == yval.iloc[:, j].shape[0]
            ypred.index = yval.iloc[:, j].index
            smp.append(smape(ypred, yval.iloc[:, j]))
        except Exception:
            smp.append(0)
        smp_sat.append(min(smp))
        smp_total.append(min(smp))
        smp = list(map(lambda x: 100*(1-x), smp))
#         print(sat_id, j, smp)
        if smp[0]>smp[1]:
            try:
                target = satY.iloc[:, j]
                li = [(target.iloc[0+i:-24+i]).reset_index(drop=True).rename('r{}'.format(24-i)) for i in range(24)]
                mean = target.rolling(24).apply(lambda x: x.mean()).rename('mean').iloc[23:-1].reset_index(drop=True)
                std = target.rolling(24).apply(lambda x: x.std()).rename('std').iloc[23:-1].reset_index(drop=True)
                li.append(mean)
                li.append(std)
                x = pd.concat(li, axis = 1)
                ya = target.shift(-24).iloc[:-24].reset_index(drop=True)

                from sklearn.linear_model import Ridge
                ridge = Ridge(alpha = 0.001, normalize=False)
                ridge.fit(x, ya)

                for i in range(satXtest.shape[0]):
                    mean = target.tail(25).rolling(24).apply(lambda x: x.mean()).rename('mean').iloc[-1]
                    std = target.tail(25).rolling(24).apply(lambda x: x.std()).rename('std').iloc[-1]
                    li = [target.iloc[-24+i] for i in range(24)]
                    li.append(mean)
                    li.append(std)
                    x = np.array(li).reshape(1, len(li))
                    ya = pd.Series(ridge.predict(x))
                    ya.index = [target.index[-1]+1]
                    target = target.append(pd.Series(ya))

                target = target[-satXtest.shape[0]:]
                target.index = satXtest.index
                submission.loc[satXtest.index, submission.columns[j+1]] = target
            except Exception as e:
                print(sat_id, j, e)
                try:
                    mod = sm.tsa.statespace.SARIMAX(satY.iloc[:, j], order=(0,0,0), seasonal_order=(1,0,1,24), simple_differencing=True)
                    res = mod.fit(disp=False)
                except Exception as e:
                    try:
                        mod = sm.tsa.statespace.SARIMAX(satY.iloc[:, j], order=(0,0,0), seasonal_order=(1,0,0,24), simple_differencing=True)
                        res = mod.fit(disp=False)
                    except Exception as e:
                        mod = sm.tsa.statespace.SARIMAX(satY.iloc[:, j], order=(0,0,0), seasonal_order=(1,0,2,24), simple_differencing=True)
                        res = mod.fit(disp=False)
                nforecast = satXtest.shape[0]
                pred = res.get_forecast(steps = nforecast)
                ypred = pred.predicted_mean
                assert len(ypred) == satXtest.iloc[:, j].shape[0]
                ypred.index = satXtest.iloc[:, j].index
                submission.loc[satXtest.index, submission.columns[j+1]] = ypred
        else:
            try:
                mod = sm.tsa.statespace.SARIMAX(satY.iloc[:, j], order=(0,0,0), seasonal_order=(1,0,1,24), simple_differencing=True)
                res = mod.fit(disp=False)
            except Exception as e:
                try:
                    mod = sm.tsa.statespace.SARIMAX(satY.iloc[:, j], order=(0,0,0), seasonal_order=(1,0,0,24), simple_differencing=True)
                    res = mod.fit(disp=False)
                except Exception as e:
                    mod = sm.tsa.statespace.SARIMAX(satY.iloc[:, j], order=(0,0,0), seasonal_order=(1,0,2,24), simple_differencing=True)
                    res = mod.fit(disp=False)
            nforecast = satXtest.shape[0]
            pred = res.get_forecast(steps = nforecast)
            ypred = pred.predicted_mean
            assert len(ypred) == satXtest.iloc[:, j].shape[0]
            ypred.index = satXtest.iloc[:, j].index
            submission.loc[satXtest.index, submission.columns[j+1]] = ypred
    print(sat_id, 100*(1- np.mean(smp_sat)), 100*(1- np.mean(smp_total)))
submission.to_csv('submission_sarima_rolling.csv', index = None)

1 92.84248058787676 92.84248058787676
2 98.29145536010073 95.56696797398875
3 97.6625577120678 96.26549788668176
4 88.66774781356662 94.36606036840298
6 92.84769754106618 94.06238780293562

9 99.21630134244721 94.9213733928542
16 97.71780204042933 95.32086319965066
20 82.02015526529055 93.65827470785565
22 85.73181808157725 92.77755730493583
24 99.29051670031852 93.4288532444741
25 86.88336593321361 92.8338089434504
26 93.16430083235203 92.86134993419222

27 83.26682299738407 92.1233094005916
28 95.7842197141247 92.38480299441538
29 99.41666013129782 92.85359347020754
32 97.20223763652642 93.12538373060248
34 99.68262977695024 93.51110408627
35 88.80392296312051 93.24959402387279
36 99.91028753704778 93.6001568403557
37 46.05677157149327 91.22298757691257
38 88.10000648353088 91.07427419151345
39 99.62015550544861 91.46272334214686
40 99.96102201540475 91.83221458881025
41 90.98002798259016 91.79670681355108

42 99.45059867413832 92.10286248797456
44 99.43162090109413 92.38473781155608

400 99.94589137815035 93.87511632391711
407 5 Input contains NaN, infinity or a value too large for dtype('float64').
407 91.56904467692242 93.86358596568213
409 97.63030182690098 93.8823258455887
411 99.20037700136182 93.90865283150836
412 86.9694945022424 93.87446978555138
415 88.55061418268683 93.8483724541648
416 88.65503908971813 93.8230391206797
417 99.82878563840173 93.85219322998903
418 89.380054961533 93.83058869729118
419 99.96734514705211 93.86009233406887
422 2 Input contains NaN, infinity or a value too large for dtype('float64').
422 91.09750262383037 93.84687420148398

423 99.78020039140924 93.87512813572172
424 99.84389443769682 93.90341612767422
426 99.85811939473942 93.93150435063208
428 99.94556601284363 93.95973938191007
435 98.70581205485941 93.98191729159674
436 99.95265329113258 94.00968815671085
437 81.80410565830495 93.95318083032936
438 87.9112798021643 93.9253379684484
440 98.03896161419013 93.94420780168575
443 82.79497823740206 93.89329807764793
446 99.9220